In [8]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


def process_file(file_path):
    # Derive names and create output directory alongside the CSV
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parent_dir = os.path.dirname(file_path)
    output_dir = os.path.join(parent_dir, f"{base_name} plots")
    os.makedirs(output_dir, exist_ok=True)

    # Read CSV, skip first two summary rows, use third row as header
    df = pd.read_csv(file_path, delimiter=',', skiprows=2, header=0)

    # 1) Mass Flow Rate plot with linear fit
    x = df['Run Time [s]']
    y = df['Weight [Kg]']
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    r_squared = r_value**2

    plt.figure()
    plt.scatter(x, y, label='Data')
    plt.plot(x, slope * x + intercept,
             label=f'Fit: y={slope:.4f}x + {intercept:.4f}\n$R^2$={r_squared:.4f}')
    plt.xlabel('Run Time [s]')
    plt.ylabel('Weight [Kg]')
    plt.title('Mass Flow Rate vs Run Time')
    plt.legend()
    plt.tight_layout()
    mass_flow_plot = os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_plot.png")
    plt.savefig(mass_flow_plot)
    plt.close()

    # 2) Differential Pressure vs Run Time
    plt.figure()
    plt.plot(df['Run Time [s]'], df['Differential Pressure [Bar]'], marker='o')
    plt.xlabel('Run Time [s]')
    plt.ylabel('Differential Pressure [Bar]')
    plt.title('Differential Pressure vs Run Time')
    plt.tight_layout()
    dp_plot = os.path.join(output_dir, f"{base_name}_Differential_Pressure_plot.png")
    plt.savefig(dp_plot)
    plt.close()

    # 3) Rotational Speed vs Run Time
    plt.figure()
    plt.plot(df['Run Time [s]'], df['Rotational Speed [RPM]'], marker='o')
    plt.xlabel('Run Time [s]')
    plt.ylabel('Rotational Speed [RPM]')
    plt.title('Rotational Speed vs Run Time')
    plt.tight_layout()
    speed_plot = os.path.join(output_dir, f"{base_name}_Rotational_Speed_plot.png")
    plt.savefig(speed_plot)
    plt.close()

    # 4) Torque vs Run Time
    plt.figure()
    plt.plot(df['Run Time [s]'], df['Torque [N-m]'], marker='o')
    plt.xlabel('Run Time [s]')
    plt.ylabel('Torque [N-m]')
    plt.title('Torque vs Run Time')
    plt.tight_layout()
    torque_plot = os.path.join(output_dir, f"{base_name}_Torque_plot.png")
    plt.savefig(torque_plot)
    plt.close()
    
    # --- Filtered plots (Weight between 20 and 30 Kg) ---
    filtered_df = df[(df['Weight [Kg]'] >= 20) & (df['Weight [Kg]'] <= 30)]
    if not filtered_df.empty:
        # Compute filtered run time span
        frt = filtered_df['Run Time [s]'].iloc[-1] - filtered_df['Run Time [s]'].iloc[0]

        # 1) Mass Flow Rate (Filtered)
        xf = filtered_df['Run Time [s]']
        yf = filtered_df['Weight [Kg]']
        slope_f, intercept_f, r_val_f, _, _ = linregress(xf, yf)
        r2_f = r_val_f**2

        plt.figure()
        plt.scatter(xf, yf, label='Filtered Data')
        plt.plot(xf, slope_f*xf + intercept_f,
                 label=f'Fit: y={slope_f:.4f}x+{intercept_f:.4f}\n$R^2$={r2_f:.4f}')
        plt.xlabel('Run Time [s]')
        plt.ylabel('Weight [Kg]')
        plt.title('Mass Flow Rate vs Run Time (Filtered)')
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s',
                 transform=plt.gca().transAxes,
                 verticalalignment='top')
        plt.legend()
        plt.ylim(bottom=0)
        plt.tight_layout()
        mf_f = os.path.join(output_dir, f"{base_name}_Mass_Flow_Rate_filtered_plot.png")
        plt.savefig(mf_f)
        plt.close()

        # 2) Differential Pressure (Filtered)
        dpf = filtered_df['Differential Pressure [Bar]']
        avg_dpf = dpf.mean()
        cnt_dpf = dpf.count()
        std_dpf = dpf.std(ddof=1)

        plt.figure()
        plt.plot(filtered_df['Run Time [s]'], dpf, marker='o')
        plt.xlabel('Run Time [s]')
        plt.ylabel('Differential Pressure [Bar]')
        plt.title('Differential Pressure vs Run Time (Filtered)')
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_dpf:.4f} Bar\n'
                 f'Count: {cnt_dpf}\n'
                 f'SD: {std_dpf:.4f}',
                 transform=plt.gca().transAxes,
                 verticalalignment='top')
        plt.ylim(bottom=0)
        plt.tight_layout()
        dp_f = os.path.join(output_dir, f"{base_name}_Differential_Pressure_filtered_plot.png")
        plt.savefig(dp_f)
        plt.close()

        # 3) Rotational Speed (Filtered)
        rsf = filtered_df['Rotational Speed [RPM]']
        avg_rsf = rsf.mean()
        cnt_rsf = rsf.count()
        std_rsf = rsf.std(ddof=1)

        plt.figure()
        plt.plot(filtered_df['Run Time [s]'], rsf, marker='o')
        plt.xlabel('Run Time [s]')
        plt.ylabel('Rotational Speed [RPM]')
        plt.title('Rotational Speed vs Run Time (Filtered)')
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_rsf:.4f} RPM\n'
                 f'Count: {cnt_rsf}\n'
                 f'SD: {std_rsf:.4f}',
                 transform=plt.gca().transAxes,
                 verticalalignment='top')
        plt.ylim(bottom=0)
        plt.tight_layout()
        rs_f = os.path.join(output_dir, f"{base_name}_Rotational_Speed_filtered_plot.png")
        plt.savefig(rs_f)
        plt.close()

        # 4) Torque (Filtered)
        tf = filtered_df['Torque [N-m]']
        avg_tf = tf.mean()
        cnt_tf = tf.count()
        std_tf = tf.std(ddof=1)

        plt.figure()
        plt.plot(filtered_df['Run Time [s]'], tf, marker='o')
        plt.xlabel('Run Time [s]')
        plt.ylabel('Torque [N-m]')
        plt.title('Torque vs Run Time (Filtered)')
        plt.text(0.05, 0.95,
                 f'Filtered Run Time: {frt:.2f} s\n'
                 f'Avg: {avg_tf:.4f} N-m\n'
                 f'Count: {cnt_tf}\n'
                 f'SD: {std_tf:.4f}',
                 transform=plt.gca().transAxes,
                 verticalalignment='top')
        plt.ylim(bottom=0)
        plt.tight_layout()
        t_f = os.path.join(output_dir, f"{base_name}_Torque_filtered_plot.png")
        plt.savefig(t_f)
        plt.close()
    else:
        print('No data in filtered weight range (20–30 Kg)')
    
    print(f"Processed {file_path}: plots saved in '{output_dir}'")


def main(csv_dir):
    # Scan directory for all CSV files
    pattern = os.path.join(csv_dir, '*.csv')
    files = glob.glob(pattern)
    if not files:
        print(f"No CSV files found in {csv_dir}")
        return
    for fp in files:
        process_file(fp)


if __name__ == '__main__':
    # Set this to your target directory containing CSV files
    csv_directory = '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025'
    main(csv_directory)

Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1503_7760RPM_27.5%_#6_0.352Kg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1503_7760RPM_27.5%_#6_0.352Kg-s plots'
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1456_7250RPM_27.5%_#14_0.331Kg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1456_7250RPM_27.5%_#14_0.331Kg-s plots'
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1449_7250RPM_28%_#1

/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10919: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10933: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10936: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


No data in filtered weight range (20–30 Kg)
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1628_7760RPM_27.5%_#10_NaNKg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1628_7760RPM_27.5%_#10_NaNKg-s plots'
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_0915_7250RPM_25.6%_#7_0.328Kg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_0915_7250RPM_25.6%_#7_0.328Kg-s plots'
No data in filtered weight range (20–30 Kg)
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior P

/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10919: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10933: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10936: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


No data in filtered weight range (20–30 Kg)
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1525_7760RPM_27.5%_#7_NaNKg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1525_7760RPM_27.5%_#7_NaNKg-s plots'
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1813_7250RPM_19%_#1_0.249Kg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1813_7250RPM_19%_#1_0.249Kg-s plots'


/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10919: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10933: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/krai/Development/Programming/myenv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:10936: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


No data in filtered weight range (20–30 Kg)
Processed /Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1134_6730RPM_25.6%_#5_NaNKg-s.csv: plots saved in '/Users/krai/Library/CloudStorage/OneDrive-ChulalongkornUniversity/Electric Pump Senior Project/Experimental Record/Glycerine/17042025/Glycerine_17042025_1134_6730RPM_25.6%_#5_NaNKg-s plots'
